In [2]:
import json
import pandas as pd
import numpy as np
from pathlib import Path
from typing import Any
import re

In [6]:
# 1. Load the data
file_path = "data/train.json"
with open(file_path, 'r') as f:
    data = json.load(f)

print(f"Dataset contains {len(data)} examples")

Dataset contains 3037 examples


In [9]:
data[0]

{'pre_text': ['26 | 2009 annual report in fiscal 2008 , revenues in the credit union systems and services business segment increased 14% ( 14 % ) from fiscal 2007 .',
  'all revenue components within the segment experienced growth during fiscal 2008 .',
  'license revenue generated the largest dollar growth in revenue as episys ae , our flagship core processing system aimed at larger credit unions , experienced strong sales throughout the year .',
  'support and service revenue , which is the largest component of total revenues for the credit union segment , experienced 34 percent growth in eft support and 10 percent growth in in-house support .',
  'gross profit in this business segment increased $ 9344 in fiscal 2008 compared to fiscal 2007 , due primarily to the increase in license revenue , which carries the highest margins .',
  'liquidity and capital resources we have historically generated positive cash flow from operations and have generally used funds generated from operations

In [17]:
import json
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

# Load both datasets
train_path = "data/train.json"
dev_path = "data/dev.json"

with open(train_path, 'r') as f:
    train_data = json.load(f)
    
with open(dev_path, 'r') as f:
    dev_data = json.load(f)

print(f"Train dataset: {len(train_data)} examples")
print(f"Dev dataset: {len(dev_data)} examples")

# Examine the structure of dev dataset
print("\nExamining dev data structure...")
dev_keys_at_root = Counter()
for ex in dev_data:
    for key in ex.keys():
        dev_keys_at_root[key] += 1

print("Keys at root level and their frequency:")
for key, count in dev_keys_at_root.most_common():
    print(f"{key}: {count} ({count/len(dev_data)*100:.2f}%)")

# Sample data structure
def print_sample_structure(data, dataset_name, num_samples=2):
    print(f"\n{dataset_name} sample data structures:")
    for i, ex in enumerate(data[:num_samples]):
        print(f"\nSample {i+1} keys: {list(ex.keys())}")
        if 'qa' in ex:
            print(f"  'qa' keys: {list(ex['qa'].keys())}")
        elif any(k.startswith('qa_') for k in ex.keys()):
            qa_keys = [k for k in ex.keys() if k.startswith('qa_')]
            print(f"  Found multiple QA keys: {qa_keys}")
            if qa_keys:
                print(f"  '{qa_keys[0]}' keys: {list(ex[qa_keys[0]].keys())}")

print_sample_structure(dev_data, "Dev")

# Functions for analysis (reusing from before but with additional comparison capabilities)
def analyze_dataset(data, dataset_name):
    """Analyze a dataset and return key statistics"""
    
    # Basic counts
    total_examples = len(data)
    examples_with_qa = sum(1 for ex in data if 'qa' in ex)
    examples_with_qa_n = sum(1 for ex in data if any(k.startswith('qa_') for k in ex.keys()))
    
    # Extract QA data
    qa_list = []
    for ex in data:
        if 'qa' in ex:
            qa_list.append(ex['qa'])
        else:
            for key in [k for k in ex.keys() if k.startswith('qa_')]:
                if key in ex and isinstance(ex[key], dict):
                    qa_list.append(ex[key])
    
    # QA stats
    questions = [qa.get('question', '') for qa in qa_list if qa]
    question_lengths = [len(q.split()) for q in questions if q]
    
    answers = [qa.get('answer', '') for qa in qa_list if qa]
    answer_types = {
        'percentage': sum('%' in str(a) for a in answers),
        'currency': sum('$' in str(a) for a in answers),
        'decimal': sum('.' in str(a) for a in answers)
    }
    
    # Steps and operations
    all_steps = []
    all_operations = []
    for qa in qa_list:
        if 'steps' in qa and qa['steps']:
            steps = qa['steps']
            all_steps.append(len(steps))
            all_operations.extend([step.get('op', '') for step in steps])
    
    # Table stats
    table_rows = [len(ex.get('table', [])) for ex in data]
    table_cols = [len(ex.get('table', [[]])[0]) if ex.get('table') and len(ex.get('table', [[]])) > 0 else 0 
                  for ex in data]
    
    # Dialogue stats
    dialogue_turns = []
    for ex in data:
        if 'annotation' in ex and 'dialogue_break' in ex['annotation']:
            dialogue_turns.append(len(ex['annotation']['dialogue_break']))
    
    return {
        'dataset_name': dataset_name,
        'total_examples': total_examples,
        'qa_examples': examples_with_qa,
        'qa_n_examples': examples_with_qa_n,
        'question_lengths': {
            'mean': np.mean(question_lengths) if question_lengths else 0,
            'min': min(question_lengths) if question_lengths else 0,
            'max': max(question_lengths) if question_lengths else 0
        },
        'answer_types': answer_types,
        'steps_count': {
            'mean': np.mean(all_steps) if all_steps else 0,
            'min': min(all_steps) if all_steps else 0,
            'max': max(all_steps) if all_steps else 0
        },
        'operation_counts': Counter(all_operations),
        'table_size': {
            'rows_mean': np.mean(table_rows) if table_rows else 0,
            'rows_max': max(table_rows) if table_rows else 0,
            'cols_mean': np.mean(table_cols) if table_cols else 0,
            'cols_max': max(table_cols) if table_cols else 0
        },
        'dialogue_turns': {
            'mean': np.mean(dialogue_turns) if dialogue_turns else 0,
            'min': min(dialogue_turns) if dialogue_turns else 0,
            'max': max(dialogue_turns) if dialogue_turns else 0
        }
    }

# Analyze both datasets
train_stats = analyze_dataset(train_data, "Train")
dev_stats = analyze_dataset(dev_data, "Dev")

# Print comparative statistics
def print_comparative_stats(train_stats, dev_stats):
    print("\n=== COMPARATIVE STATISTICS ===")
    print(f"{'Statistic':<30} {'Train':<15} {'Dev':<15}")
    print("-" * 60)
    print(f"{'Total examples':<30} {train_stats['total_examples']:<15} {dev_stats['total_examples']:<15}")
    print(f"{'Examples with qa':<30} {train_stats['qa_examples']:<15} {dev_stats['qa_examples']:<15}")
    print(f"{'Examples with qa_n':<30} {train_stats['qa_n_examples']:<15} {dev_stats['qa_n_examples']:<15}")
    
    print("\n--- Question & Answer ---")
    print(f"{'Mean question length':<30} {train_stats['question_lengths']['mean']:.2f}{'':<10} {dev_stats['question_lengths']['mean']:.2f}")
    
    print("\n--- Answer Types ---")
    for answer_type in ['percentage', 'currency', 'decimal']:
        train_pct = train_stats['answer_types'][answer_type] / train_stats['total_examples'] * 100
        dev_pct = dev_stats['answer_types'][answer_type] / dev_stats['total_examples'] * 100
        print(f"{answer_type.capitalize():<30} {train_pct:.2f}%{'':<9} {dev_pct:.2f}%")
    
    print("\n--- Reasoning Steps ---")
    print(f"{'Mean steps per question':<30} {train_stats['steps_count']['mean']:.2f}{'':<10} {dev_stats['steps_count']['mean']:.2f}")
    print(f"{'Max steps per question':<30} {train_stats['steps_count']['max']}{'':<10} {dev_stats['steps_count']['max']}")
    
    print("\n--- Table Size ---")
    print(f"{'Mean table rows':<30} {train_stats['table_size']['rows_mean']:.2f}{'':<10} {dev_stats['table_size']['rows_mean']:.2f}")
    print(f"{'Mean table columns':<30} {train_stats['table_size']['cols_mean']:.2f}{'':<10} {dev_stats['table_size']['cols_mean']:.2f}")
    
    print("\n--- Dialogue ---")
    print(f"{'Mean dialogue turns':<30} {train_stats['dialogue_turns']['mean']:.2f}{'':<10} {dev_stats['dialogue_turns']['mean']:.2f}")

print_comparative_stats(train_stats, dev_stats)

# Compare top operations
def compare_operations(train_stats, dev_stats):
    print("\n=== TOP OPERATIONS COMPARISON ===")
    train_ops = train_stats['operation_counts']
    dev_ops = dev_stats['operation_counts']
    
    # Get all unique operations
    all_ops = set(list(train_ops.keys()) + list(dev_ops.keys()))
    
    # Calculate percentages
    train_total = sum(train_ops.values())
    dev_total = sum(dev_ops.values())
    
    print(f"{'Operation':<15} {'Train %':<15} {'Dev %':<15} {'Difference':<15}")
    print("-" * 60)
    
    # Sort by total frequency
    sorted_ops = sorted(all_ops, key=lambda op: (train_ops.get(op, 0) + dev_ops.get(op, 0)), reverse=True)
    
    for op in sorted_ops[:10]:  # Top 10 operations
        train_pct = train_ops.get(op, 0) / train_total * 100 if train_total else 0
        dev_pct = dev_ops.get(op, 0) / dev_total * 100 if dev_total else 0
        diff = dev_pct - train_pct
        
        print(f"{op:<15} {train_pct:.2f}%{'':<10} {dev_pct:.2f}%{'':<10} {diff:+.2f}%")

compare_operations(train_stats, dev_stats)

# Check for potential data leakage/overlap
def check_overlap(train_data, dev_data):
    print("\n=== DATA OVERLAP ANALYSIS ===")
    
    # Check question overlap
    train_questions = set()
    dev_questions = set()
    
    for ex in train_data:
        if 'qa' in ex and 'question' in ex['qa']:
            train_questions.add(ex['qa']['question'])
        for key in [k for k in ex.keys() if k.startswith('qa_')]:
            if key in ex and 'question' in ex[key]:
                train_questions.add(ex[key]['question'])
    
    for ex in dev_data:
        if 'qa' in ex and 'question' in ex['qa']:
            dev_questions.add(ex['qa']['question'])
        for key in [k for k in ex.keys() if k.startswith('qa_')]:
            if key in ex and 'question' in ex[key]:
                dev_questions.add(ex[key]['question'])
    
    question_overlap = train_questions.intersection(dev_questions)
    
    # Check file/document overlap
    train_files = set(ex.get('filename', '') for ex in train_data)
    dev_files = set(ex.get('filename', '') for ex in dev_data)
    file_overlap = train_files.intersection(dev_files)
    
    print(f"Question overlap: {len(question_overlap)} questions ({len(question_overlap)/len(dev_questions)*100:.2f}% of dev)")
    print(f"Document overlap: {len(file_overlap)} files ({len(file_overlap)/len(dev_files)*100:.2f}% of dev)")
    
    if len(question_overlap) > 0:
        print("\nSample overlapping questions:")
        for q in list(question_overlap)[:3]:
            print(f"- {q}")

check_overlap(train_data, dev_data)

Train dataset: 3037 examples
Dev dataset: 421 examples

Examining dev data structure...
Keys at root level and their frequency:
pre_text: 421 (100.00%)
post_text: 421 (100.00%)
filename: 421 (100.00%)
table_ori: 421 (100.00%)
table: 421 (100.00%)
id: 421 (100.00%)
annotation: 421 (100.00%)
qa: 300 (71.26%)
qa_0: 121 (28.74%)
qa_1: 121 (28.74%)

Dev sample data structures:

Sample 1 keys: ['pre_text', 'post_text', 'filename', 'table_ori', 'table', 'qa', 'id', 'annotation']
  'qa' keys: ['question', 'answer', 'explanation', 'ann_table_rows', 'ann_text_rows', 'steps', 'program', 'gold_inds', 'exe_ans', 'program_re']

Sample 2 keys: ['pre_text', 'post_text', 'filename', 'table_ori', 'table', 'id', 'annotation', 'qa_0', 'qa_1']
  Found multiple QA keys: ['qa_0', 'qa_1']
  'qa_0' keys: ['question', 'answer', 'explanation', 'ann_table_rows', 'ann_text_rows', 'steps', 'program', 'gold_inds', 'exe_ans', 'program_re']

=== COMPARATIVE STATISTICS ===
Statistic                      Train          

## Multi-question Example

In [19]:
import json

# Load the data
file_path = "/Users/richardyang/Desktop/Long Term Career/UK 2025 Cases/TomoroAI/FinQA_Experim/data/data/train.json"
with open(file_path, 'r') as f:
    data = json.load(f)

# Find a multi-QA example
multi_qa_example = None
for example in data:
    if 'qa_0' in example and 'qa_1' in example:
        multi_qa_example = example
        break

if multi_qa_example:
    print("Example ID:", multi_qa_example.get('id', 'Unknown'))
    print("\nSource:", multi_qa_example.get('filename', 'Unknown'))
    
    # Print the questions and answers
    print("\nQuestion 1:", multi_qa_example['qa_0']['question'])
    print("Answer 1:", multi_qa_example['qa_0']['answer'])
    
    print("\nQuestion 2:", multi_qa_example['qa_1']['question'])
    print("Answer 2:", multi_qa_example['qa_1']['answer'])
    
    # Print the dialogue structure if available
    if 'dialogue_break_ori' in multi_qa_example.get('annotation', {}):
        print("\nDialogue structure:")
        for i, turn in enumerate(multi_qa_example['annotation']['dialogue_break_ori']):
            print(f"Turn {i+1}: {turn}")

Example ID: Double_UPS/2009/page_33.pdf

Source: UPS/2009/page_33.pdf

Question 1: what is the roi of an investment in ups in 2004 and sold in 2006?
Answer 1: -8.9%

Question 2: what was the difference in percentage cumulative return on investment for united parcel service inc . compared to the s&p 500 index for the five year period ended 12/31/09?
Answer 2: -26.16%

Dialogue structure:
Turn 1: what was the fluctuation of the performance price of the ups from 2004 to 2006?
Turn 2: and how much does this fluctuation represent in relation to that price in 2004?
Turn 3: and from this year to 2009, what was the fluctuation for that stock?
Turn 4: what is this fluctuation as a percentage of the 2004 price?
Turn 5: and for the s&p 500 index price, what was the fluctuation in those five years?
Turn 6: and what percentage does this fluctuation represent in relation to the 2004 price of this stock?
Turn 7: what is, then, the difference between the ups percentage and this s&p 500 index one, for 

# Creating the datasets for Training 

In [ ]:
with open(train_path, 'r') as f:
    train_data = json.load(f)
    
with open(dev_path, 'r') as f:
    dev_data = json.load(f)

print(f"Loaded {len(train_data)} training examples and {len(dev_data)} dev examples")

# Helper functions
def format_table_to_text(table: list[list[str]]) -> str:
    """Convert table to a readable text format"""
    if not table or len(table) == 0:
        return ""
    
    formatted_rows = []
    
    # Add header
    header = " | ".join(str(cell) for cell in table[0])
    formatted_rows.append(header)
    formatted_rows.append("-" * len(header))  # separator line
    
    # Add data rows
    for row in table[1:]:
        formatted_rows.append(" | ".join(str(cell) for cell in row))
    
    return "\n".join(formatted_rows)

def clean_text(text_list: list[str]) -> str:
    """Clean and join text segments"""
    if not text_list:
        return ""
    return " ".join([re.sub(r'\s+', ' ', segment.strip()) for segment in text_list])

def format_program_steps(steps: list[dict]) -> str:
    """Format program steps into a human-readable chain of thought"""
    if not steps:
        return ""
    
    formatted_steps = []
    
    for i, step in enumerate(steps, 1):
        operation = step.get('op', '')
        arg1 = step.get('arg1', '')
        arg2 = step.get('arg2', '')
        result = step.get('res', '')
        
        # Clean up operation name for readability
        op_name = operation
        if operation.startswith('divide'):
            op_name = 'divide'
        elif operation.startswith('minus'):
            op_name = 'subtract'
        elif operation.startswith('add'):
            op_name = 'add'
        elif operation.startswith('multiply'):
            op_name = 'multiply'
            
        formatted_step = f"Step {i}: {op_name}({arg1}, {arg2}) = {result}"
        formatted_steps.append(formatted_step)
        
    return "\n".join(formatted_steps)

def get_qa_pairs(example: dict) -> list[dict]:
    """Extract all QA pairs from an example (handles both single and multi-QA formats)"""
    qa_pairs = []
    
    # Handle regular QA
    if 'qa' in example and isinstance(example['qa'], dict):
        qa_pairs.append(example['qa'])
    
    # Handle multi-QA (qa_0, qa_1, etc.)
    qa_indices = sorted([int(k[3:]) for k in example.keys() if k.startswith('qa_') and k[3:].isdigit()])
    for idx in qa_indices:
        key = f'qa_{idx}'
        if key in example and isinstance(example[key], dict):
            qa_pairs.append(example[key])
    
    return qa_pairs

# Process function for Objective A (Final Answer Only)
def process_for_final_answer(data: list[dict], split_name: str) -> pd.DataFrame:
    """Process data for the final answer only objective"""
    processed_data = []
    
    for example in data:
        # Get context elements
        pre_text = clean_text(example.get('pre_text', []))
        post_text = clean_text(example.get('post_text', []))
        table_text = format_table_to_text(example.get('table', []))
        example_id = example.get('id', '')
        
        # Get all QA pairs
        qa_pairs = get_qa_pairs(example)
        
        for qa_idx, qa in enumerate(qa_pairs):
            if not qa or 'question' not in qa or 'answer' not in qa:
                continue
                
            question = qa['question']
            answer = qa['answer']
            
            # Create prompt
            prompt = f"""### Context:
{pre_text}

### Table:
{table_text}

### Additional Context:
{post_text}

### Question:
{question}

### Response:
"""
            
            # Create unique ID for this QA pair
            qa_id = f"{example_id}_{qa_idx}" if qa_idx > 0 else example_id
            
            processed_data.append({
                'prompt': prompt,
                'answer': answer,
                'id': qa_id,
                'split': split_name
            })
    
    return pd.DataFrame(processed_data)

# Process function for Objective B (Program + Answer)
def process_for_program_answer(data: list[dict], split_name: str) -> pd.DataFrame:
    """Process data for the program + answer objective"""
    processed_data = []
    
    for example in data:
        # Get context elements
        pre_text = clean_text(example.get('pre_text', []))
        post_text = clean_text(example.get('post_text', []))
        table_text = format_table_to_text(example.get('table', []))
        example_id = example.get('id', '')
        
        # Get all QA pairs
        qa_pairs = get_qa_pairs(example)
        
        for qa_idx, qa in enumerate(qa_pairs):
            if not qa or 'question' not in qa or 'answer' not in qa or 'steps' not in qa:
                continue
                
            question = qa['question']
            answer = qa['answer']
            steps = qa['steps']
            
            # Create prompt
            prompt = f"""### Context:
{pre_text}

### Table:
{table_text}

### Additional Context:
{post_text}

### Question:
{question}

### Response:
<think>
"""
            
            # Format program steps
            program_text = format_program_steps(steps)
            full_output = f"""{program_text}
</think>
Final Answer: {answer}"""
            
            # Create unique ID for this QA pair
            qa_id = f"{example_id}_{qa_idx}" if qa_idx > 0 else example_id
            
            processed_data.append({
                'prompt': prompt,
                'output': full_output,
                'id': qa_id,
                'split': split_name
            })
    
    return pd.DataFrame(processed_data)

# Process both datasets for both objectives
print("Processing datasets...")

# Objective A: Final Answer Only
train_final_answer = process_for_final_answer(train_data, "train")
dev_final_answer = process_for_final_answer(dev_data, "dev")
final_answer_df = pd.concat([train_final_answer, dev_final_answer])

# Objective B: Program + Answer
train_program_answer = process_for_program_answer(train_data, "train")
dev_program_answer = process_for_program_answer(dev_data, "dev")
program_answer_df = pd.concat([train_program_answer, dev_program_answer])

# Create output directory
output_dir = Path("data/processed_datasets")
output_dir.mkdir(exist_ok=True, parents=True)

# Save datasets
final_answer_df.to_csv(output_dir / "finqa_final_answer.csv", index=False)
final_answer_df.to_json(output_dir / "finqa_final_answer.jsonl", orient='records', lines=True)

program_answer_df.to_csv(output_dir / "finqa_program_answer.csv", index=False)
program_answer_df.to_json(output_dir / "finqa_program_answer.jsonl", orient='records', lines=True)

# Save train/dev splits separately
train_final_answer.to_json(output_dir / "finqa_final_answer_train.jsonl", orient='records', lines=True)
dev_final_answer.to_json(output_dir / "finqa_final_answer_dev.jsonl", orient='records', lines=True)

train_program_answer.to_json(output_dir / "finqa_program_answer_train.jsonl", orient='records', lines=True)
dev_program_answer.to_json(output_dir / "finqa_program_answer_dev.jsonl", orient='records', lines=True)

# Print statistics about the processed datasets
print(f"\nProcessed {len(final_answer_df)} examples for Objective A (Final Answer Only)")
print(f"  - Train: {len(train_final_answer)} examples")
print(f"  - Dev: {len(dev_final_answer)} examples")

print(f"\nProcessed {len(program_answer_df)} examples for Objective B (Program + Answer)")
print(f"  - Train: {len(train_program_answer)} examples")
print(f"  - Dev: {len(dev_program_answer)} examples")

print(f"\nAll processed datasets saved to {output_dir}")

Loaded 3037 training examples and 421 dev examples
Processing datasets...

Processed 4507 examples for Objective A (Final Answer Only)
  - Train: 3965 examples
  - Dev: 542 examples

Processed 4507 examples for Objective B (Program + Answer)
  - Train: 3965 examples
  - Dev: 542 examples

All processed datasets saved to data/processed_datasets
